## PyCaret 설치

In [ ]:
!pip install pycaret

Google Colab 사용자의 경우 다음의 코드를 실행합니다.

In [ ]:
from pycaret.utils import enable_colab

enable_colab()

## 필요한 모듈 import

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

pd.options.display.max_columns = None

## 실습을 위한 데이터셋 로드

In [ ]:
dataset = sns.load_dataset('titanic')
dataset.head()

In [ ]:
# 데이터셋 크기 출력
dataset.shape

In [ ]:
train = dataset.sample(frac=0.8, random_state=786)
test = dataset.drop(train.index)
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)
print('학습용 데이터셋: ' + str(train.shape))
print('예측용 데이터셋: ' + str(test.shape))

### 컬럼 설명

- survivied: 생존여부 (1: 생존, 0: 사망)
- pclass: 좌석 등급 (1등급, 2등급, 3등급)
- sex: 성별
- age: 나이
- sibsp: 형제 + 배우자 수
- parch: 부모 + 자녀 수
- fare: 좌석 요금
- embarked: 탑승 항구 (S, C, Q)
- class: pclass와 동일
- who: 성별과 동일
- adult_male: 성인 남자 여부
- deck: 데크 번호 (알파벳 + 숫자 혼용)
- embark_town: 탑승 항구 이름
- alive: 생존여부 (yes, no)
- alone: 혼자 탑승 여부

## 1-1) 셋업 setup

머신러닝 예측 방식에 따라 다음 중 하나를 골라 import 합니다.

In [ ]:
from pycaret.classification import *    # 분류
# from pycaret.regression import *      # 회귀
# from pycaret.clustering import *      # 군집
# from pycaret.anomaly import *         # 이상 탐지
# from pycaret.arules import *          # 연관 규칙

**`setup` 함수**

- `data`: 학습할 데이터셋을 지정합니다.
- `target`: 예측할 대상(target) 컬럼을 지정합니다.
- `session_id`: SEED 값을 지정합니다.

- `profile`: True로 설정시 데이터 프로파일링을 출력합니다.

In [ ]:
train.head(1)

`ignore_features`에는 분석 / 학습에 무시할 컬럼을 지정합니다.

- `alive`, `embark_town`, `class`는 다른 컬럼과 겹치는 컬럼이므로 제외합니다.

In [ ]:
ignore_features=['alive', 'embark_town', 'class']

`categorical_features`에는 범주형 컬럼을 지정합니다.
- `pclass`, `sex`, `embarked` 컬럼은 범주형 특성을 가집니다. 즉, 카테고리화 할 수 있는 컬럼을 의미합니다.

In [ ]:
categorical_features=['pclass', 'sex', 'embarked']

`numeric_features`에는 수치형 컬럼을 지정합니다.
- `age`, `fare`, `sibsp`, `parch`는 숫자형태의 데이터를 가지므로 수치형 컬럼에 지정합니다.

In [ ]:
numeric_features=['age', 'fare', 'sibsp', 'parch']

In [ ]:
clf = setup(data=train, 
            target='survived', 
            ignore_features=ignore_features,           # 분석/학습에 고려하지 않을 feature(컬럼) 제거
            categorical_features=categorical_features, # 범주형 컬럼 지정
            numeric_features=numeric_features,         # 수치형 컬럼 지정
            session_id=123, 
            silent=True,
            ) 

## 1-2) 고급 데이터 전처리

[문서 링크](https://pycaret.readthedocs.io/en/latest/api/classification.html)

### `categorical_imputation`: str, default = ‘constant’

범주형 데이터에 대한 결측치를 어떻게 채워줄지에 대한 전략을 지정합니다. 기본 값은 `constant`이며 `constant`로 지정시 **not_available**로 채워지게 됩니다.

`mode`로 변경시 최빈 값으로 채워지게 됩니다.

In [ ]:
clf = setup(data=train, 
            target='survived', 
            ignore_features=ignore_features,           # 분석/학습에 고려하지 않을 feature(컬럼) 제거
            categorical_features=categorical_features, # 범주형 컬럼 지정
            numeric_features=numeric_features,         # 수치형 컬럼 지정
            categorical_imputation='mode',             # 범주형 결측치를 최빈값으로 채움
            session_id=123, 
            silent=True,
            ) 

`high_cardinality_features`: list of str, default = None

범주형의 데이터 종류(level)가 많은 경우 우리는 몇 가지의 대분류로 압축하여 데이터를 단순화 할 수 있습니다.
```
(예시)
United-States                 29170
Mexico                          643
?                               583
Philippines                     198
Germany                         137
Canada                          121
Puerto-Rico                     114
El-Salvador                     106
India                           100
Cuba                             95
England                          90
Jamaica                          81
South                            80
China                            75
Italy                            73
Dominican-Republic               70
Vietnam                          67
Guatemala                        64
Japan                            62
Poland                           60
Columbia                         59
Taiwan                           51
Haiti                            44
Iran                             43
Portugal                         37
Nicaragua                        34
Peru                             31
France                           29
Greece                           29
Ecuador                          28
Ireland                          24
Hong                             20
Cambodia                         19
Trinadad&Tobago                  19
Laos                             18
Thailand                         18
Yugoslavia                       16
Outlying-US(Guam-USVI-etc)       14
Honduras                         13
Hungary                          13
Scotland                         12
Holand-Netherlands                1
Name: native_country, dtype: int64
```

위의 국가별 범주를 살펴봤을 때 너무 많은 범주가 존재하기 때문에 효율적인 학습이 어렵습니다.

우리는 임의의 그룹을 만들어 단순화할 수 있습니다.

```
income_01 = ['Jamaica',
 'Haiti',
 'Puerto-Rico',
 'Laos',
 'Thailand',
 'Ecuador',]

income_02 = ['Outlying-US(Guam-USVI-etc)',
 'Honduras',
 'Columbia',
 'Dominican-Republic',
 'Mexico',
 'Guatemala',
 'Portugal',
 'Trinadad&Tobago',
 'Nicaragua',
 'Peru',
 'Vietnam',
 'El-Salvador',]

income_03 = ['Poland',
 'Ireland',
 'South',
 'China',]

income_04 = [
    'United-States',
]
income_05 = [
 'Greece',
 'Scotland',
 'Cuba',
 'Hungary',
 'Hong',
 'Holand-Netherlands',
]
income_06 = [
 'Philippines',
 'Canada',
]
income_07 = [
 'England',
 'Germany',
]

income_08 = [
 'Italy',
 'India',
 'Japan',
 'France',
 'Yugoslavia',
 'Cambodia',
]

income_09 = [
 'Taiwan',
 'Iran',
]

income_other=['?', ]
```

```
(결과)
income_04       29170
income_02        1157
income_other      583
income_06         319
income_01         303
income_08         299
income_03         239
income_07         227
income_05         170
income_09          94
Name: country_bin, dtype: int64
```

`high_cardinality_features`에 리스트 형식으로 컬럼을 지정해주면 됩니다.

### imputation_type: str, default = ‘simple’

`imputation`은 결측치를 채우는 전략입니다.

`imputation_type`은 기본 `simple`이 지정되어 있어, 결측단순 값으로 치환됩니다. 예를 들면 **평균**, **중앙값**과 같은 단순 값으로 말이죠.

하지만, 머신러닝 알고리즘을 활용하여 예측하여 값을 채워 줄 수 있습니다. 적용하기 위해서는 `imputation_type='iterative'`로 설정합니다.

`iterative_imputation_iters`은 iteration 횟수를 지정하는데, 기본 5가 설정되어 있습니다.
- 너무 숫자를 늘리게 되면 과적합될 수 있습니다.

In [ ]:
clf = setup(data=train, 
            target='survived', 
            ignore_features=ignore_features,           # 분석/학습에 고려하지 않을 feature(컬럼) 제거
            categorical_features=categorical_features, # 범주형 컬럼 지정
            numeric_features=numeric_features,         # 수치형 컬럼 지정
            categorical_imputation='mode',             # 범주형 결측치를 최빈값으로 채움
            imputation_type='iterative',               # 결측치를 lightgbm으로 예측하여 채움
            iterative_imputation_iters=10,             # imputation iteration 지정
            session_id=123, 
            silent=True,
            ) 

### normalize / normalize_method

- `normalize`: bool, default = False
- `normalize_method`: str, default = ‘zscore’

`normalize`는 데이터의 정규화/표준화 여부를 설정합니다.

`normalize_method`는 기본 'zscore'가 설정되어 있습니다.

- 'minmax': scales and translates each feature individually such that it is in the range of 0 - 1.
- 'maxabs': scales and translates each feature individually such that the maximal absolute value of each feature will be 1.0.
- 'robust': scales and translates each feature according to the Interquartile range. When the dataset contains outliers, robust scaler often gives better results.

In [ ]:
clf = setup(data=train, 
            target='survived', 
            ignore_features=ignore_features,           # 분석/학습에 고려하지 않을 feature(컬럼) 제거
            categorical_features=categorical_features, # 범주형 컬럼 지정
            numeric_features=numeric_features,         # 수치형 컬럼 지정
            categorical_imputation='mode',             # 범주형 결측치를 최빈값으로 채움
            imputation_type='iterative',               # 결측치를 lightgbm으로 예측하여 채움
            iterative_imputation_iters=10,             # imputation iteration 지정
            normalize=True,                            # 정규화 적용
            normalize_method='zscore',                 # 정규화 방식 지정
            session_id=123, 
            silent=True,
            ) 

## 2-1) 모든 모델에 대한 학습 compare_models

`compare_models`
- `sort`: 정렬 기준이 되는 평가지표를 설정합니다.
- `n_select`: 상위 N개의 알고리즘을 선택합니다.
- `fold`: Cross Validation 평가 Fold의 개수를 지정합니다.

In [ ]:
best_models = compare_models(sort='Accuracy', n_select=3, fold=5)

### 모델 블렌딩 blend_models

- `compare_models`로 추출된 best 모델에 대하여 모델 블렌딩하여 성능 개선
- `Soft` voting 방식으로 `estimator_list`에 적용된 모델을 앙상블
- `Voting Ensemble`

In [ ]:
blended_models = blend_models(best_models, fold=5)

## 2-2) 단일 모델 생성 create_model / 배깅 앙상블 ensemble_

단일 모델을 생성하기 위해서는 `create_model`로 생성할 수 있습니다.

단일 모델 리스트
- `lr` - Logistic Regression
- `knn` - K Neighbors Classifier
- `nb` - Naive Bayes
- `dt` - Decision Tree Classifier
- `svm` - SVM - Linear Kernel
- `rbfsvm` - SVM - Radial Kernel
- `gpc` - Gaussian Process Classifier
- `mlp` - MLP Classifier
- `ridge` - Ridge Classifier
- `rf` - Random Forest Classifier
- `qda` - Quadratic Discriminant Analysis
- `ada` - Ada Boost Classifier
- `gbc` - Gradient Boosting Classifier
- `lda` - Linear Discriminant Analysis
- `et` - Extra Trees Classifier
- `xgboost` - Extreme Gradient Boosting
- `lightgbm` - Light Gradient Boosting Machine
- `catboost` - CatBoost Classifier

In [ ]:
dt = create_model('dt')

### 배깅 앙상블 ensemble_model

In [ ]:
ensembled_models = ensemble_model(dt)

## 3) 모델 예측 predict_model

- `Label`에 예측된 결과를 확인할 수 있습니다.
- `Score`에 예측된 결과의 **확률** 값을 확인할 수 있습니다.

In [ ]:
predict_model(data=dataset, estimator=ensembled_models)

## 4) 모델 분석 interpret_model


### 특성 중요도 Feature Importances

각 특성별 종속변수(Y)에 미치는 영향도를 계산

In [ ]:
plot_model(dt, plot='feature')

### 오차행렬 confusion_matrix

In [ ]:
from IPython.display import Image

Image('https://skappal7.files.wordpress.com/2018/08/confusion-matrix.jpg', width=750)

In [ ]:
plot_model(dt, plot='confusion_matrix')

**정밀도 Precision**

- TP / (FP + TP)
- 중요 사례: 스팸 분류 (스팸으로 분류한 메일 중에서 중요 메일이 있는 경우
                
**재현율 Recall**

- TP / (FN + TP)
- 중요 사례: 암 진단 (암 환자를 건강으로 분류하는 경우)

### 섀플리 분석 SHAP(SHapley Additive exPlanations)

- 게임 이론에서 처음 등장하였으며, 머신러닝 모델을 분석하기 시작하면서 다시 주목받고 있음
- 섀플리 값 (SHAP Value)는 각각의 데이터가 종속변수 (Y)에 미치는 영향도를 계산한 값 (기여도를 수치화)


In [ ]:
# 섀플리 설치
!pip install shap

In [ ]:
interpret_model(dt)

In [ ]:
interpret_model(dt, plot='reason')